# Basic Configs

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt

## Global Variables

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cpu = torch.device("cpu")
batch_size = 128
num_workers = 4

## Load Repo
- Rename the folder to `project`

In [ ]:
!git clone https://github.com/effie-0/IDL-Project.git

Cloning into 'IDL-Project'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 193 (delta 89), reused 100 (delta 35), pack-reused 0
Receiving objects: 100% (193/193), 1.59 MiB | 3.38 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [ ]:
!mv IDL-Project project

# Load Functions

In [ ]:
from project.summarize.load_data import AdvDataset, get_data

## Load Dataset

In [ ]:
trainset, trainloader, testset, testloader = get_data(batch_size)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Load Models

In [ ]:
from project.summarize.classifier import ResidualBlock, ResNet18, cifar_resnet20, ResNet50, VGG

In [ ]:
resnet18 = ResNet18(ResidualBlock)
resnet20 = cifar_resnet20('cifar10')  # With pretrained model
resnet50 = ResNet50()
vgg16 = VGG('VGG16')

Downloading: "https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar10-resnet20-30abc31d.pth" to /root/.cache/torch/hub/checkpoints/cifar10-resnet20-30abc31d.pth


### Load Trained Models

In [ ]:
vgg16_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/vgg16.pth"
vgg16.load_state_dict(torch.load(vgg16_path)['model_state_dict'])

resnet18_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/ResNet18.pth"
resnet18.load_state_dict(torch.load(resnet18_path)['model_state_dict'])

resnet50_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/Copy of ResNet50_49.pth"
resnet50.load_state_dict(torch.load(resnet50_path)['model_state_dict'])

<All keys matched successfully>

In [ ]:
models = {
    'vgg16': vgg16,
    'resnet18': resnet18,
    'resnet20': resnet20,
    'resnet50': resnet50,
}

# Generate the Adversarial Samples

In [ ]:
from project.summarize.generate_attack import generate_fgsm_attack

In [ ]:
criterion = nn.CrossEntropyLoss()

## Evaluate the models

In [ ]:
from project.summarize.normal_train import evaluate

In [ ]:
eps = 0.0

for model_name, model in models.items():
    model.to(device)
    model.eval()
    acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
    loss, acc = evaluate(model, testloader, criterion, device)
    print('eval acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

Epsilon: 0.0	Test Accuracy = 9231 / 10000 = 0.9231
eval acc =  92.31
Epsilon: 0.0	Test Accuracy = 9054 / 10000 = 0.9054
eval acc =  90.53999999999999
Epsilon: 0.0	Test Accuracy = 9165 / 10000 = 0.9165
eval acc =  91.64999999999999
Epsilon: 0.0	Test Accuracy = 9081 / 10000 = 0.9081
eval acc =  90.81


## Eps = 0.05

In [ ]:
adv_samples = {}
eps = 0.05

for model_name, model in models.items():
    model.to(device)
    model.eval()
    acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
    model.to(cpu)
    torch.cuda.empty_cache()
    dataset = AdvDataset(adv_ex, adv_label)
    loader = data.DataLoader(dataset, batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)
    adv_samples[model_name] = (dataset, loader)

Epsilon: 0.05	Test Accuracy = 2012 / 10000 = 0.2012
Epsilon: 0.05	Test Accuracy = 2244 / 10000 = 0.2244
Epsilon: 0.05	Test Accuracy = 2545 / 10000 = 0.2545
Epsilon: 0.05	Test Accuracy = 2475 / 10000 = 0.2475


## Eps = 0.03

In [ ]:
other_samples = {}
eps = 0.03

for model_name, model in models.items():
    model.to(device)
    model.eval()
    acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
    model.to(cpu)
    torch.cuda.empty_cache()
    dataset = AdvDataset(adv_ex, adv_label)
    loader = data.DataLoader(dataset, batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)
    other_samples[model_name] = (dataset, loader)

Epsilon: 0.03	Test Accuracy = 3783 / 10000 = 0.3783
Epsilon: 0.03	Test Accuracy = 4352 / 10000 = 0.4352
Epsilon: 0.03	Test Accuracy = 3391 / 10000 = 0.3391
Epsilon: 0.03	Test Accuracy = 4059 / 10000 = 0.4059


# 4x4 Matrix

In [ ]:
matrix = {}
for model_name, model in models.items():
    line = {}
    matrix[model_name] = line
    model.to(device)
    for adv_name, (dataset, loader) in adv_samples.items():
        print('classifier: ', model_name, ';  adversarial data comes from: ', adv_name)
        loss, acc = evaluate(model, loader, criterion, device)
        line[adv_name] = acc
        print('acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

classifier:  vgg16 ;  adversarial data comes from:  vgg16
acc =  20.119999999999997
classifier:  vgg16 ;  adversarial data comes from:  resnet18
acc =  75.94999999999999
classifier:  vgg16 ;  adversarial data comes from:  resnet20
acc =  81.71000000000001
classifier:  vgg16 ;  adversarial data comes from:  resnet50
acc =  76.73
classifier:  resnet18 ;  adversarial data comes from:  vgg16
acc =  78.83
classifier:  resnet18 ;  adversarial data comes from:  resnet18
acc =  22.439999999999998
classifier:  resnet18 ;  adversarial data comes from:  resnet20
acc =  82.73
classifier:  resnet18 ;  adversarial data comes from:  resnet50
acc =  76.13
classifier:  resnet20 ;  adversarial data comes from:  vgg16
acc =  71.75
classifier:  resnet20 ;  adversarial data comes from:  resnet18
acc =  71.86
classifier:  resnet20 ;  adversarial data comes from:  resnet20
acc =  25.45
classifier:  resnet20 ;  adversarial data comes from:  resnet50
acc =  65.53
classifier:  resnet50 ;  adversarial data comes

In [ ]:
matrix = {}
for model_name, model in models.items():
    line = {}
    matrix[model_name] = line
    model.to(device)
    for adv_name, (dataset, loader) in other_samples.items():
        print('classifier: ', model_name, ';  adversarial data comes from: ', adv_name)
        loss, acc = evaluate(model, loader, criterion, device)
        line[adv_name] = acc
        print('acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

classifier:  vgg16 ;  adversarial data comes from:  vgg16
acc =  37.830000000000005
classifier:  vgg16 ;  adversarial data comes from:  resnet18
acc =  84.8
classifier:  vgg16 ;  adversarial data comes from:  resnet20
acc =  86.99
classifier:  vgg16 ;  adversarial data comes from:  resnet50
acc =  84.89
classifier:  resnet18 ;  adversarial data comes from:  vgg16
acc =  84.26
classifier:  resnet18 ;  adversarial data comes from:  resnet18
acc =  43.519999999999996
classifier:  resnet18 ;  adversarial data comes from:  resnet20
acc =  86.33999999999999
classifier:  resnet18 ;  adversarial data comes from:  resnet50
acc =  83.39999999999999
classifier:  resnet20 ;  adversarial data comes from:  vgg16
acc =  81.32000000000001
classifier:  resnet20 ;  adversarial data comes from:  resnet18
acc =  82.03
classifier:  resnet20 ;  adversarial data comes from:  resnet20
acc =  33.910000000000004
classifier:  resnet20 ;  adversarial data comes from:  resnet50
acc =  77.38000000000001
classifier: